# Implementation of RFC 2104 - HMAC Algorithm
___Hash-based Message Authentication___ or ___Key Hashed Message Authentication Algorithm___

...in python

* [RFC 2104](https://www.ietf.org/rfc/rfc2104.txt)
* [Wikipedia](https://en.wikipedia.org/wiki/HMAC)
* [Dr. Dobb's](http://www.drdobbs.com/security/the-hmac-algorithm/184410908)
* [RFC 2202](https://tools.ietf.org/html/rfc2202.html) - test cases for RFC 2104.

Here is an implementation of the RFC 2104 (HMAC Algorithm) in python. This is a demonstration of the working of the HMAC algorithm. This code is not production grade. For one, this code does not handle the case of $key\: length > Block\: Size$

In [1]:
import hashlib
import hmac

In [2]:
### A few test cases from RFC2202
# [hash_algo, key, key_len, data, reference hash]
testCase=[
    ['md5',0x0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b,16,"Hi There",8,0x9294727a3638bb1c13f48ef8158bfc9d],
    ['md5',"Jefe",4,"what do ya want for nothing?",28,0x750c783e6ab0b503eaa86e310a5db738],
    ['md5',0x0c0c0c0c0c0c0c0c0c0c0c0c0c0c0c0c,16,"Test With Truncation",20,0x56461ef2342edc00f9bab995690efd4c],
    ['sha1',0x0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b,20,"Hi There",8,0xb617318655057264e28bc0b6fb378c8ef146be00],
    ['sha1',"Jefe",4,"what do ya want for nothing?",28,0xeffcdf6ae5eb2fa2d27416d5f184df9c259a7c79],
    ['sha1',0x0c0c0c0c0c0c0c0c0c0c0c0c0c0c0c0c0c0c0c0c,20,"Test With Truncation",20,0x4c1a03424b55e07fe7f27be1d58bb9324a9a5a04]
]

In [3]:
### Define key, message, Block size, hashing algorithm 
B = 64
# Choose one of the test cases from above and transform the key and message from string/int to bytes
use_test_case = 5
hash_algo = testCase[use_test_case][0]
if isinstance(testCase[use_test_case][1],bytes):
    key = testCase[use_test_case][1]
elif isinstance(testCase[use_test_case][1],str):
    key = testCase[use_test_case][1].encode()
else:
    key = (testCase[use_test_case][1]).to_bytes(testCase[use_test_case][2],byteorder='big')

if isinstance(testCase[use_test_case][3],bytes):
    message = testCase[use_test_case][3]
else:
    message = testCase[use_test_case][3].encode()
ref_hash = testCase[use_test_case][5]

In [4]:
### hash (message) - Just to make the code easy to read
def hash(M):
    return hashlib.new(hash_algo, M).digest()

In [5]:
### XOR Translation table
trans_5C = bytes((x ^ 0x5C) for x in range(256))
trans_36 = bytes((x ^ 0x36) for x in range(256))

The frist step is to zero pad the key $K$ to the match the block size. The zero padding is done on the right. This zero padded key is refered to as $K'$. $K'$ is XORed with the inner pad (ipad) sequence (0x36363636 ... block length). $K'$ is also XORed with the outer pad (opad) sequence (0x5C5C5C ... block length)
![Algorithm](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/SHAhmac.svg/400px-SHAhmac.svg.png)Image from wikipedia

In [6]:
### Zero pad key to block length B
# Ideally one should check to see if the key length > block size.
# Zero pad the key on the right.
K_zpad=key.ljust(B,b'\0')
# Xor with ipad and opad sequence
K_ipad=K_zpad.translate(trans_36)
K_opad=K_zpad.translate(trans_5C)

# The paded key now looks like this...
print("Key XOR ipad(0x363636...):\n"+K_ipad.hex())
print("\n\nKey XOR opad(0x505050...):\n"+K_opad.hex())

Key XOR ipad(0x363636...):
3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3636363636363636363636363636363636363636363636363636363636363636363636363636363636363636


Key XOR opad(0x505050...):
50505050505050505050505050505050505050505c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c


The message is appended to $K' \bigoplus ipad$ and hashed.

In [7]:
### Append message
part1=K_ipad+message
print("key XOR ipad || message:\n"+part1.hex())

key XOR ipad || message:
3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3a3636363636363636363636363636363636363636363636363636363636363636363636363636363636363636546573742057697468205472756e636174696f6e


$hash( ( K' \bigoplus ipad ) \parallel message )$

In [8]:
### hash (key xor ipad || message )
hash1=hash(part1)
print("hash( key XOR ipad || message):\n"+hash1.hex())

hash( key XOR ipad || message):
ed4916d25d214f8be71e13e93e63c62061fb5862


The hash from the previous step is appended to the $K' \bigoplus opad$ and hashed again. This hash is the HMAC hash.

$ hash( K' \bigoplus opad \parallel hash( ( K' \bigoplus ipad ) \parallel message ) ) $

In [9]:
### hash (key xor opad || hash ( (key xor ipad)||message ) )
part2 = K_opad + hash1
print("(key KOR opad) || hash((key XOR ipad)||message):\n0x"+part2.hex())
hash_final = hash(part2)
print("\nFinal Hash:\n0x"+hash_final.hex())

# Now verify our calculated hash with the hash from the test case
print("\nReference Hash:\n"+hex(ref_hash))

(key KOR opad) || hash((key XOR ipad)||message):
0x50505050505050505050505050505050505050505c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5ced4916d25d214f8be71e13e93e63c62061fb5862

Final Hash:
0x4c1a03424b55e07fe7f27be1d58bb9324a9a5a04

Reference Hash:
0x4c1a03424b55e07fe7f27be1d58bb9324a9a5a04


## Compare with Python's HMAC Implementation

In [10]:
# Python
pyH=hmac.new(key,message,digestmod=hash_algo)
print("0x"+pyH.hexdigest())

0x4c1a03424b55e07fe7f27be1d58bb9324a9a5a04
